In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [39]:
yellow_tripdata_2023_01 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
yellow_tripdata_2023_02 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
yellow_tripdata_2023_01.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


### Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [4]:
yellow_tripdata_2023_01.shape[1]

19

### Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [5]:
yellow_tripdata_2023_01['duration'] =  yellow_tripdata_2023_01.tpep_dropoff_datetime - yellow_tripdata_2023_01.tpep_pickup_datetime

yellow_tripdata_2023_01.duration = yellow_tripdata_2023_01.duration.apply(lambda x: x.total_seconds() / 60)

In [7]:
yellow_tripdata_2023_01.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [8]:
yellow_tripdata_2023_01.duration.std()

42.594351241920904

### Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [9]:
pd.set_option('display.float_format', '{:.3f}'.format)

In [9]:
yellow_tripdata_2023_01.duration.describe(percentiles=[0.90, 0.92, 0.95, 0.98])

count   3066766.000
mean         15.669
std          42.594
min         -29.200
50%          11.517
90%          27.942
92%          30.550
95%          36.467
98%          48.733
max       10029.183
Name: duration, dtype: float64

In [10]:
yellow_tripdata_2023_01[((yellow_tripdata_2023_01.duration >= 1) & (yellow_tripdata_2023_01.duration <= 60))].shape[0] / len(yellow_tripdata_2023_01.duration)

0.9812202822125979

In [12]:
yellow_tripdata_2023_01 =  yellow_tripdata_2023_01[(yellow_tripdata_2023_01.duration >= 1) & (yellow_tripdata_2023_01.duration <= 60)]

### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [13]:
dv = DictVectorizer()

In [14]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [15]:
yellow_tripdata_2023_01[categorical] = yellow_tripdata_2023_01[categorical].astype(str)

In [16]:
yellow_tripdata_2023_01.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.000,0.970,1.000,N,161,141,2,9.300,1.000,0.500,0.000,0.000,1.000,14.300,2.500,0.000,8.433
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.000,1.100,1.000,N,43,237,1,7.900,1.000,0.500,4.000,0.000,1.000,16.900,2.500,0.000,6.317
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.000,2.510,1.000,N,48,238,1,14.900,1.000,0.500,15.000,0.000,1.000,34.900,2.500,0.000,12.750


In [17]:
yellow_tripdata_2023_01.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [21]:
train_dicts  = yellow_tripdata_2023_01[categorical].to_dict(orient='records')

In [24]:
X_train = dv.fit_transform(train_dicts)

In [25]:
len(dv.feature_names_)

515

### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [26]:
lr = LinearRegression()

In [27]:
y_train = yellow_tripdata_2023_01.duration

In [28]:
model = lr.fit(X_train, y_train)

In [29]:
y_pred = model.predict(X_train)

In [30]:
mean_squared_error(y_train, y_pred, squared=False)

7.649262059978435

### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [45]:
yellow_tripdata_2023_02['duration'] =  yellow_tripdata_2023_02.tpep_dropoff_datetime - yellow_tripdata_2023_02.tpep_pickup_datetime

yellow_tripdata_2023_02.duration = yellow_tripdata_2023_02.duration.apply(lambda x: x.total_seconds() / 60)

In [46]:
yellow_tripdata_2023_02.duration.describe()

count   2913955.000
mean         16.016
std          42.842
min         -43.617
25%           7.250
50%          11.800
75%          18.767
max        7053.617
Name: duration, dtype: float64

In [47]:
yellow_tripdata_2023_02 =  yellow_tripdata_2023_02[(yellow_tripdata_2023_02.duration >= 1) & (yellow_tripdata_2023_02.duration <= 60)]

In [48]:
yellow_tripdata_2023_02[categorical] = yellow_tripdata_2023_02[categorical].astype(str)

In [49]:
val_dicts = yellow_tripdata_2023_02[categorical].to_dict(orient='records')

In [50]:
X_train_2 = dv.transform(val_dicts)

In [51]:
y_pred_2 = model.predict(X_train_2)

In [54]:
mean_squared_error(yellow_tripdata_2023_02.duration, y_pred_2, squared=False)

7.811818442402467